In [26]:
import pandas as pd
from tensorflow import keras
import numpy as np
from collections import Counter

In [27]:
songs = pd.read_csv('songs_preprocessed.csv')
songs.head()

,genre,song_id,artist_id,song_name,artist_name,lyric
0,발라드,34431086,839736,취중고백,김민석 (멜로망스),뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들...
1,발라드,34481682,261143,겨울잠,아이유,때 이른 봄 몇 송이 꺾어다 너의 방 문 앞에 두었어 긴 잠 실컷 자고 나오면 그때...
2,발라드,34061322,994944,사랑은 늘 도망가,임영웅,눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...
3,발라드,34360855,2138620,눈이 오잖아(Feat.헤이즈),이무진,한 달 좀 덜 된 기억들 주머니에 넣은 채 걷고 있어 몇 시간을 혹시 몰라 네가 좋...
4,발라드,33496587,2863470,다정히 내 이름을 부르면,경서예지,끝없이 별빛이 내리던 밤 기분 좋은 바람이 두 빰을 스치고 새벽 바다 한곳을 보는 ...


# 텐서플로 데이터셋 객체를 만들기

In [28]:
target = songs['genre']
data = songs['lyric']
ds_raw = data.values
ds_target = target.values

# 토크나이즈 하고 고유 토큰 갯수 확인

In [29]:
import konlpy
from konlpy.tag import Okt
okt = Okt()

In [30]:
def tokenize(text):
  return okt.morphs(text)

In [31]:
ds_raw[0]

'뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들 만나서 오랜만에 술을 좀 했는데 자꾸만 니 얼굴 떠올라 무작정 달려왔어 이 맘 모르겠니 요즘 난 미친 사람처럼 너만 생각해 대책없이 네가 점점 좋아져 아냐 안 취했어 진짜야 널 정말 사랑해 눈물이 날만큼 원하고 있어 정말로 몰랐니 가끔 전화해 장난치듯 주말엔 뭐할거냐며 너의 관심 끌던 나를 그리고 한번씩 누나 주려 샀는데 너 그냥 준다고 생색 낸 선물도 너 때문에 산거야 이 맘 모르겠니 요즘 난 미친 사람처럼 너만 생각해 대책없이 네가 점점 좋아져 아냐 안 취했어 진짜야 널 정말 사랑해 진심이야 믿어줘 갑자기 이런 말 놀랐다면 미안해 부담이 되는게 당연해 이해해 널 하지만 내 고백도 이해해 주겠니 oh 지금 당장 대답하진마 나와 일주일만 사귀어줄래 후회없이 잘 해주고 싶은데 그 후에도 니가 싫다면 나 그때 포기할게 귀찮게 안할게 혼자 아플게 진심이야 너를 사랑하고 있어 '

In [32]:
tokens = [tokenize(d) for d in ds_raw]

for i in range(len(ds_raw)):
  ds_raw[i] = tokenize(ds_raw[i])

In [35]:
token_counts = Counter([tk for d in ds_raw for tk in d]) # 중복제거 토큰들
token_counts.most_common(10)

[('이', 11246),
 ('을', 7445),
 ('에', 7261),
 ('내', 7107),
 ('가', 6543),
 ('너', 6343),
 ('i', 5494),
 ('나', 5175),
 ('의', 5056),
 ('you', 3927)]

# 고유 토큰을 정수로 인코딩

In [39]:
encoder = {tk: i for i, tk in enumerate(token_counts.keys())}
len(encoder)
# encoder 확인
encoder['you']

588

# 데이터를 정수로 변환하고 train, test, valid split

In [43]:
from sklearn.model_selection import train_test_split
# train test split
ds_train, ds_test, target_train, target_test = train_test_split(ds_raw, ds_target, test_size=0.2, random_state=42)
# validation split
ds_train, ds_val, target_train, target_val = train_test_split(ds_train, target_train, test_size=0.2, random_state=42)

In [46]:
# encode
ds_train = [np.array([encoder[tk] for tk in d]) for d in ds_train]
ds_val = [np.array([encoder[tk] for tk in d]) for d in ds_val]
ds_test = [np.array([encoder[tk] for tk in d]) for d in ds_test]

In [34]:
for i in range(len(ds_train)):
  ds_train[i] = tokenize(ds_train[i])
for i in range(len(ds_test)):
  ds_test[i] = tokenize(ds_test[i])
for i in range(len(ds_val)):
  ds_val[i] = tokenize(ds_val[i])

AssertionError: phrase input should be string, not <class 'list'>